### test old and new average table

In [ ]:
import pandas as pd
previous_avg=pd.read_csv("../output/variant_scoring/alpha_control/vanRheene_2021_250kb_from_lead_averaged.variant_scores.tsv",\
           sep='\t')

In [11]:
average_table=pd.read_csv("../output/variant_scoring/alpha_control/vanRheene_2021_250kb_from_lead.mean.variant_scores.tsv",\
                         sep='\t')

In [25]:
merged_table=pd.merge(previous_avg,average_table,on='variant_id')

In [28]:
for i, row in merged_table.iterrows():
    assert row['logfc'] - row['logfc.mean'] < 0.001
    assert row['jsd'] - row['jsd.mean'] < 0.001
    assert row['abs_logfc'] - row['abs_logfc.mean'] < 0.001

### check coding vs noncoding table

In [1]:
import pandas as pd
### pval <= 5e-8



In [47]:
def get_lead_variant_table(flank_size=250000):
    gwas_results = pd.read_csv("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/sod1_project/variant_file/vanRheene_2021_summary_stats_hg38.csv")
    lead_variants=[['chr9',27563868,'rs2453555'],
                  ['chr19',17752689,'rs12608932'],
                  ['chr21',33039603,'rs80265967'],
                  ['chr14',31045596,'rs229195'],
                  ['chr3',39508968,'rs631312'],
                  ['chr6',32672641,'rs9275477'],
                  ['chr12',57975700,'rs113247976'],
                  ['chr21',45753117,'rs75087725'],
                  ['chr5',150410835,'rs10463311'],
                   ['chr20',48438761,'rs17785991'],
                   ['chr12',64877053,'rs4075094'],
                   ['chr5',172354731,'rs517339'],
                   ['chr4',170583157,'rs62333164'],
                   ['chr13',46113984,'rs2985994'],
                   ['chr7',157481780,'rs10280711']]

    coding_variants=[['chr21',33039603,'rs80265967'],
                    ['chr12',57975700,'rs113247976'],
                    ['chr21',45753117,'rs75087725']]

    noncoding_variants=[['chr14',31045596,'rs229195'],
                       ['chr3',39508968,'rs631312'],
                       ['chr5',150410835,'rs10463311'],
                       ['chr20',48438761,'rs17785991'],
                       ['chr5',172354731,'rs517339'],
                       ['chr13',46113984,'rs2985994'],
                       ['chr7',157481780,'rs10280711']]

    total_filtered_results=[]
    for chromo,hg17_pos,rsid in lead_variants:
        row = gwas_results[gwas_results['rsid'] == rsid]
        assert chromo == row['chr'].values
        assert hg17_pos == row['pos'].values

        pos = row['start'].values[0]
        pos = pos - 1

        ### get all variants +/- 250kb from lead variants
        start = pos - flank_size
        end = pos + flank_size
        filtered_results = gwas_results[(gwas_results['chr'] == chromo) & (gwas_results['start'] > start) & (gwas_results['end'] < end)]
        filtered_results['lead_variant'] = rsid
        total_filtered_results += [filtered_results]
    lead_variant_table=pd.concat(total_filtered_results)[['rsid','lead_variant']]
    lead_variant_table=lead_variant_table.rename({'rsid':'variant_id'},axis=1)
    return lead_variant_table

In [267]:
average_table=pd.read_csv("../output/variant_scoring/alpha_control/vanRheene_2021_250kb_from_lead.mean.variant_scores.tsv",\
                         sep='\t')

In [268]:
average_table=pd.merge(average_table,lead_variant_table,on=['variant_id'])
len(average_table)

29575

In [270]:
significant_variants=average_table[(average_table['logfc.mean.pval'] < 0.01)]

In [271]:
len(significant_variants)

198

In [272]:
total_coding_var, significant_coding_var = 0,0
for _,_,var in coding_variants:
    total_coding_var += len(average_table[average_table['lead_variant'] == var])
    significant_coding_var += len(significant_variants[significant_variants['lead_variant'] == var])


In [273]:
total_coding_var,significant_coding_var

(5111, 29)

In [274]:
total_noncoding_var, significant_noncoding_var = 0,0
for _,_,var in noncoding_variants:
    total_noncoding_var += len(average_table[average_table['lead_variant'] == var])
    significant_noncoding_var += len(significant_variants[significant_variants['lead_variant'] == var])


In [275]:
total_noncoding_var,significant_noncoding_var

(13896, 92)

In [276]:
data = [[total_coding_var-significant_coding_var,total_noncoding_var-significant_noncoding_var],
       [significant_coding_var,significant_noncoding_var]]

In [277]:
data=pd.DataFrame(data)

In [282]:
data.columns=['coding','noncoding']

In [284]:
data.index=['non-significant','significant']

In [285]:
data

,coding,noncoding
non-significant,5082,13804
significant,29,92


In [286]:
import scipy.stats as stats
odd_ratio, p_value = stats.fisher_exact(data)
print(odd_ratio, p_value)

1.1679373295096873 0.5372808717201982


In [287]:
data

,coding,noncoding
non-significant,5082,13804
significant,29,92


### test using human in vitro data

In [83]:
average_table=pd.read_csv("../output/variant_scoring/answer_als/vanRheene_2021_250kb_from_lead.mean.variant_scores.tsv",\
                         sep='\t')

lead_variant_table = get_lead_variant_table(10000)
average_table=pd.merge(average_table,lead_variant_table,on=['variant_id'])

/tmp/ipykernel_291438/1781069196.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_results['lead_variant'] = rsid
/tmp/ipykernel_291438/1781069196.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_results['lead_variant'] = rsid
/tmp/ipykernel_291438/1781069196.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [84]:

significant_variants=average_table[(average_table['logfc.mean.pval'] < 0.05)]

In [85]:
total_coding_var, significant_coding_var = 0,0
for _,_,var in coding_variants:
    total_coding_var += len(average_table[average_table['lead_variant'] == var])
    significant_coding_var += len(significant_variants[significant_variants['lead_variant'] == var])

total_noncoding_var, significant_noncoding_var = 0,0
for _,_,var in noncoding_variants:
    total_noncoding_var += len(average_table[average_table['lead_variant'] == var])
    significant_noncoding_var += len(significant_variants[significant_variants['lead_variant'] == var])


In [86]:
total_noncoding_var,significant_noncoding_var

(656, 21)

In [87]:
data = [[total_coding_var-significant_coding_var,total_noncoding_var-significant_noncoding_var],
       [significant_coding_var,significant_noncoding_var]]
data=pd.DataFrame(data)
data.columns=['coding','noncoding']
data.index=['non-significant','significant']

In [88]:
import scipy.stats as stats
odd_ratio, p_value = stats.fisher_exact(data)
print(odd_ratio, p_value)
data

0.9788976377952756 1.0


,coding,noncoding
non-significant,148,635
significant,5,21
